In [26]:
from sqlalchemy import create_engine
import pandas as pd

In [27]:

engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')


In [28]:
table_names = engine.table_names()

In [29]:
print(table_names)

['Bookings', 'Facilities', 'Members']


In [30]:
members = pd.read_sql_query("SELECT * FROM Members", engine)
bookings = pd.read_sql_query("SELECT * FROM Bookings", engine)
facilities = pd.read_sql_query("SELECT * FROM Facilities", engine)

In [108]:
members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [109]:
bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [110]:
facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


**Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!**


In [62]:
query = """
SELECT * 
FROM(SELECT s1.facility,SUM(s1.cost) AS total_revenue
    FROM(SELECT f.name AS facility,
    CASE WHEN b.memid =0 
        THEN f.guestcost * b.slots
        ELSE f.membercost * b.slots 
    END AS cost
        FROM bookings as b
        INNER JOIN facilities AS f ON b.facid = f.facid
        INNER JOIN members AS m ON b.memid = m.memid)s1
     GROUP BY s1.facility)s2
    WHERE s2.total_revenue < 1000
"""
total_revenue = pd.read_sql_query(query, engine)
total_revenue

,facility,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


**Produce a report of members and who recommended them in alphabetic surname,firstname order**

In [97]:
query = """

SELECT 
    sub2.member_name AS Member, 
    sub2.recommender_first || ' ' || sub2.recommender_last AS Recommended_by 
FROM 
    (
      SELECT 
          sub1.member_name AS member_name, 
          sub1.recommenderId AS memberId, 
          m.firstname AS recommender_first, 
          m.surname AS recommender_last 
      FROM 
          (SELECT 
                m2.memid AS memberId, 
                m1.firstname || ' ' || m1.surname AS member_name, 
                m2.recommendedby AS recommenderId 
            FROM members AS m1 
            INNER JOIN Members AS m2 
            ON m1.memid = m2.memid 
            WHERE m2.recommendedby !='') AS sub1 
      LEFT JOIN members AS m 
      ON sub1.recommenderId = m.memid 
      WHERE m.memid != '') AS sub2
ORDER BY sub2.recommender_last;

"""
pd.read_sql_query(query, engine)

,Member,Recommended_by
0,Ramnaresh Sarwin,Florence Bader
1,Joan Coplin,Timothy Baker
2,Matthew Genting,Gerald Butters
3,Timothy Baker,Jemima Farrell
4,David Pinker,Jemima Farrell
5,Henrietta Rumney,Matthew Genting
6,Douglas Jones,David Jones
7,Nancy Dare,Janice Joplette
8,David Jones,Janice Joplette
9,John Hunt,Millicent Purview


In [96]:
query = """

SELECT 
    sub2.member_name AS Member, 
    sub2.recommender_first || ' ' || sub2.recommender_last AS Recommended_by 
FROM 
    (
      SELECT 
          sub1.member_name AS member_name, 
          sub1.recommenderId AS memberId, 
          m.firstname AS recommender_first, 
          m.surname AS recommender_last 
      FROM 
          (
            SELECT 
                m1.memid AS memberId, 
                m1.firstname || ' ' || m1.surname AS member_name, 
                m1.recommendedby AS recommenderId 
            FROM members AS m1 
            INNER JOIN Members AS m2 
                ON m1.memid = m2.memid 
            WHERE m2.recommendedby !='') AS sub1 
      LEFT JOIN members AS m 
           ON sub1.recommenderId = m.memid 
      WHERE m.memid != '') AS sub2
ORDER BY sub2.recommender_last;

"""
pd.read_sql_query(query, engine)

,Member,Recommended_by
0,Ramnaresh Sarwin,Florence Bader
1,Joan Coplin,Timothy Baker
2,Matthew Genting,Gerald Butters
3,Timothy Baker,Jemima Farrell
4,David Pinker,Jemima Farrell
5,Henrietta Rumney,Matthew Genting
6,Douglas Jones,David Jones
7,Nancy Dare,Janice Joplette
8,David Jones,Janice Joplette
9,John Hunt,Millicent Purview


**Find the facilities with their usage by member, but not guests **

In [123]:
query = """
SELECT facilities.name AS Facility, sum(bookings.slots) AS Usage
FROM bookings
LEFT JOIN facilities ON bookings.facid = facilities.facid
LEFT JOIN members on members.memid = bookings.memid
WHERE members.memid != 0
GROUP BY Facility
"""
pd.read_sql_query(query, engine)

,Facility,Usage
0,Badminton Court,1086
1,Massage Room 1,884
2,Massage Room 2,54
3,Pool Table,856
4,Snooker Table,860
5,Squash Court,418
6,Table Tennis,794
7,Tennis Court 1,957
8,Tennis Court 2,882


**Q13: Find the facilities usage by month, but not guests **


In [133]:
query = """

SELECT s.month as Month,s.facility AS Facility, SUM(s.slotsnumber) AS Usage
FROM (SELECT strftime('%m', starttime) AS month,facilities.name AS facility, bookings.slots AS slotsnumber
    FROM bookings
    LEFT JOIN facilities ON bookings.facid = facilities.facid
    LEFT JOIN members on members.memid = bookings.memid
    WHERE members.memid != 0) s
GROUP BY Month,Facility



"""
pd.read_sql_query(query, engine)

,Month,Facility,Usage
0,07,Badminton Court,165
1,07,Massage Room 1,166
2,07,Massage Room 2,8
3,07,Pool Table,110
4,07,Snooker Table,140
5,07,Squash Court,50
6,07,Table Tennis,98
7,07,Tennis Court 1,201
8,07,Tennis Court 2,123
9,08,Badminton Court,414
